In [1]:
import tensorflow as tf

C:\Users\Anton\Anaconda3\envs\da2018\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

In [3]:
def maybe_download(data_root, cifar_file_name):
    """Download the file if it does not exist."""

    dest_file_name = os.path.join(data_root, cifar_file_name)

    if not os.path.exists(dest_file_name):
        urlretrieve(url + cifar_file_name, dest_file_name)

    statinfo = os.stat(dest_file_name)
    print("File ", dest_file_name, " size : ", statinfo.st_size, " bytes.")

    return dest_file_name


def maybe_extract(data_root, cifar_file_name, file_name):
    """Extract compressed file if is was not uncomressed before."""

    tar = tarfile.open(file_name)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()

    dest_dir_name = os.path.join(data_root, cifar_dir_name)
    if not os.path.exists(dest_dir_name):
        raise Exception("No data directory. Check that archive with data exists, not corrupted and contains ",
                        cifar_dir_name, " inside.")

    return dest_dir_name


def preprocess_data(X, m, v):
    if m is None:
        m = np.mean(X, axis=0)
    if v is None:
        v = np.var(X, axis=0)
    X = (X - m) / v
    return X, m, v


def preprocess_labels(y, num_classes):
    y_gt = np.zeros((len(y), num_classes))
    for i in range(0, len(y)):
        y_gt[i, y[i]] = 1
    return y_gt

def compute_accuracy(logits, labels):
  predictions = tf.argmax(logits, axis=1, output_type=tf.int64)
  labels = tf.cast(labels, tf.int64)
  batch_size = int(logits.shape[0])
  return tf.reduce_sum(
      tf.cast(tf.equal(predictions, labels), dtype=tf.float32)) / batch_size


In [4]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import pickle
from six.moves.urllib.request import urlretrieve


In [5]:
    # GLOBAL PARAMETERS
    # Data
    cifar_file_name = "cifar-10-python.tar.gz"
    url = 'https://www.cs.toronto.edu/~kriz/'
    data_root = '.' # Change me to store data elsewhere
    cifar_dir_name = "cifar-10-batches-py"  # where cifar data is stored locally
    # Learning
    alpha = 0.5  # learning rate
    num_epochs = 20  # number of learning epochs
    batch_size = 1000  # mini-batch size for gradient descent

    data_file_name = maybe_download(data_root, cifar_file_name)
    data_dir_name = maybe_extract(data_root, cifar_file_name, data_file_name)

    # load train and test data
    train_batches = ["data_batch_1",
                     "data_batch_2",
                     "data_batch_3",
                     "data_batch_4",
                     "data_batch_5",
                     ]

    train_labels = []
    train_data = []
    for train_batch in train_batches:
        path_to_batch = data_dir_name + "/" + train_batch
        print("Loading ", path_to_batch, "...")
        batch = pickle.load(open(path_to_batch, "rb"), encoding="bytes")
        # batch is a dictionary, dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
        # let's store data and labels for all batches in a separate arrays
        train_labels.extend(batch[b"labels"])
        batch_data = batch[b"data"]
        channel_size = batch_data.shape[1] // 3
        red_channel = batch_data[:, 0:channel_size]
        green_channel = batch_data[:, channel_size:2*channel_size]
        blue_channel = batch_data[:, 2*channel_size:]
        # batch_data = np.concatenate((red_channel, green_channel, blue_channel, red_channel * green_channel,
        #                              red_channel * blue_channel, green_channel * blue_channel), axis=1)
        batch_data = np.concatenate((red_channel, green_channel, blue_channel), axis=1)

        train_data.extend(batch_data)
        print("Done.")

    # also load titles for labels
    raw = pickle.load(open(data_dir_name + "/batches.meta", "rb"))
    class_names = [x for x in raw["label_names"]]

    print(class_names)

    print(len(train_data))

    # Now let's train a very simple baseline classification model - one-vs-all multiclass logistic regression
    num_classes = len(class_names)

    train_data = np.array(train_data, dtype=float)
    train_labels = np.array(train_labels)

    print("Data shape : ", train_data.shape)
    print("Labels shape : ", train_labels.shape)

    num_features = train_data.shape[1]
    num_samples = train_data.shape[0]


File  .\cifar-10-python.tar.gz  size :  170498071  bytes.
Loading  .\cifar-10-batches-py/data_batch_1 ...
Done.
Loading  .\cifar-10-batches-py/data_batch_2 ...
Done.
Loading  .\cifar-10-batches-py/data_batch_3 ...
Done.
Loading  .\cifar-10-batches-py/data_batch_4 ...
Done.
Loading  .\cifar-10-batches-py/data_batch_5 ...
Done.
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
50000
Data shape :  (50000, 3072)
Labels shape :  (50000,)


In [6]:
class CifarClassifier(tf.keras.Model):
    
    def __init__(self, layers_data, initializer, use_batch_norm):
        super(CifarClassifier, self).__init__()
        
        self.net_layers = []
        
        for i in range(len(layers_data)):
            self.net_layers.append(tf.keras.layers.Dense(units=layers_data[i], 
                                                         activation=tf.keras.activations.sigmoid,
                                                         kernel_initializer=initializer))
            if use_batch_norm and (i != len(layers_data) - 1):
                self.net_layers.append( tf.keras.layers.BatchNormalization())
        
        self._layers += self.net_layers
        self.predict(np.random.rand(1, 3072))
        
    def call(self, inputs, **kwargs):
        x = inputs
        
        for layer in self.net_layers:
            x = layer(x)
        
        return x

In [7]:
def train(model, optimizer, train_data, train_labels):
    
    train_data, _, _ = preprocess_data(train_data, None, None)
    train_logits = preprocess_labels(train_labels, 10)
    
    acc = np.zeros(num_epochs)
    
    for epoch in range(num_epochs):
        permutation_inds = np.random.permutation(num_samples)
        train_data_epoch = train_data[permutation_inds, :]
        train_logits_epoch = train_logits[permutation_inds, :]

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            batch_x = train_data_epoch[batch_start: batch_end, :]
            batch_logits = train_logits_epoch[batch_start: batch_end, :]

            with tf.GradientTape() as tape:
                batch_logits_pred = model(batch_x)
                loss_val = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(batch_logits, 
                                                                                   batch_logits_pred))
                grads = tape.gradient(loss_val, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))

        logits_pred = model(train_data)
        acc[epoch] = compute_accuracy(logits_pred, train_labels)

        print("\nEpoch", epoch, "accuracy", acc[epoch])
        
    return acc

In [17]:
model = CifarClassifier([50, 10], 
                        initializer="random_uniform",
                        use_batch_norm=False)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha)

acc1 = train(model, optimizer, train_data, train_labels)

# [model.variables[0] + model.variables[1]]


Epoch 0 accuracy 0.1467999964952469

Epoch 1 accuracy 0.1462000012397766

Epoch 2 accuracy 0.14577999711036682

Epoch 3 accuracy 0.14579999446868896

Epoch 4 accuracy 0.14535999298095703

Epoch 5 accuracy 0.1453399956226349

Epoch 6 accuracy 0.14504000544548035

Epoch 7 accuracy 0.14509999752044678

Epoch 8 accuracy 0.14521999657154083

Epoch 9 accuracy 0.14509999752044678

Epoch 10 accuracy 0.1451600044965744

Epoch 11 accuracy 0.1451999992132187

Epoch 12 accuracy 0.1451600044965744

Epoch 13 accuracy 0.14511999487876892

Epoch 14 accuracy 0.14504000544548035

Epoch 15 accuracy 0.14508000016212463

Epoch 16 accuracy 0.1450600028038025

Epoch 17 accuracy 0.145019993185997

Epoch 18 accuracy 0.145019993185997

Epoch 19 accuracy 0.14511999487876892


In [9]:
model = CifarClassifier([50, 50, 50, 50, 10], 
                        initializer="random_uniform",
                        use_batch_norm=False)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)

acc2 = train(model, optimizer, train_data, train_labels)


Epoch 0 accuracy 0.10000000149011612

Epoch 1 accuracy 0.10000000149011612

Epoch 2 accuracy 0.10000000149011612

Epoch 3 accuracy 0.10000000149011612

Epoch 4 accuracy 0.10000000149011612

Epoch 5 accuracy 0.10000000149011612

Epoch 6 accuracy 0.10000000149011612

Epoch 7 accuracy 0.10000000149011612

Epoch 8 accuracy 0.10000000149011612

Epoch 9 accuracy 0.10000000149011612

Epoch 10 accuracy 0.10000000149011612

Epoch 11 accuracy 0.10000000149011612

Epoch 12 accuracy 0.10000000149011612

Epoch 13 accuracy 0.10000000149011612

Epoch 14 accuracy 0.10000000149011612

Epoch 15 accuracy 0.10000000149011612

Epoch 16 accuracy 0.10000000149011612

Epoch 17 accuracy 0.10000000149011612

Epoch 18 accuracy 0.10000000149011612

Epoch 19 accuracy 0.10000000149011612


In [10]:
model = CifarClassifier([50, 50, 50, 50, 10], 
                        initializer="he_normal",
                        use_batch_norm=False)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)

acc3 = train(model, optimizer, train_data, train_labels)


Epoch 0 accuracy 0.10000000149011612

Epoch 1 accuracy 0.10000000149011612

Epoch 2 accuracy 0.10000000149011612

Epoch 3 accuracy 0.10000000149011612

Epoch 4 accuracy 0.10000000149011612

Epoch 5 accuracy 0.10000000149011612

Epoch 6 accuracy 0.10000000149011612

Epoch 7 accuracy 0.10000000149011612

Epoch 8 accuracy 0.10000000149011612

Epoch 9 accuracy 0.10000000149011612

Epoch 10 accuracy 0.1024399995803833

Epoch 11 accuracy 0.10000000149011612

Epoch 12 accuracy 0.0814799964427948

Epoch 13 accuracy 0.10010000318288803

Epoch 14 accuracy 0.10000000149011612

Epoch 15 accuracy 0.10530000180006027

Epoch 16 accuracy 0.10000000149011612

Epoch 17 accuracy 0.10927999764680862

Epoch 18 accuracy 0.10000000149011612

Epoch 19 accuracy 0.10000000149011612


In [11]:
model = CifarClassifier([50, 50, 50, 50, 10], 
                        initializer="he_normal",
                        use_batch_norm=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)

acc3 = train(model, optimizer, train_data, train_labels)


Epoch 0 accuracy 0.10000000149011612

Epoch 1 accuracy 0.10000000149011612

Epoch 2 accuracy 0.10000000149011612

Epoch 3 accuracy 0.10000000149011612

Epoch 4 accuracy 0.10000000149011612

Epoch 5 accuracy 0.10016000270843506

Epoch 6 accuracy 0.1098800003528595

Epoch 7 accuracy 0.0938199982047081

Epoch 8 accuracy 0.10000000149011612

Epoch 9 accuracy 0.10000000149011612

Epoch 10 accuracy 0.09679999947547913

Epoch 11 accuracy 0.10000000149011612

Epoch 12 accuracy 0.10000000149011612

Epoch 13 accuracy 0.10000000149011612

Epoch 14 accuracy 0.10000000149011612

Epoch 15 accuracy 0.10000000149011612

Epoch 16 accuracy 0.10000000149011612

Epoch 17 accuracy 0.10000000149011612

Epoch 18 accuracy 0.10000000149011612

Epoch 19 accuracy 0.10000000149011612
